In [12]:
import pyodbc
import qrcode
import io
from PIL import Image


# Define connection parameters
server = 'DESKTOP-7L2TS2S\\SQLEXPRESS'  # Double backslashes for Python
database = 'Church'  # Replace with your actual database name
driver = '{ODBC Driver 17 for SQL Server}'  # Check your ODBC drivers

# Use Windows Authentication (Trusted Connection)
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

try:
    # Connect to SQL Server
    conn = pyodbc.connect(conn_str)
    print("✅ Connected to SQL Server successfully!")

    # Create a cursor
    cursor = conn.cursor()

    # Example: Fetch all tables in the database
    cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES")
    
    # Print results
    for row in cursor.fetchall():
        print(row)

    # Close the connection
    conn.close()
except Exception as e:
    print("❌ Connection failed:", e)


✅ Connected to SQL Server successfully!
('QR',)


In [ ]:

def generate_qr(data):
    """Generate QR code and return it as binary data."""
    qr = qrcode.make(data)
    buffered = io.BytesIO()
    qr.save(buffered, format="PNG")  # Save as PNG format
    return buffered.getvalue()  # Return binary data

def insert_data(name, four_digit_number):
    """Insert name, ID, and QR code into the database as binary."""
    try:
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()

        qr_binary = generate_qr(str(four_digit_number))  # Generate binary QR code

        # Use `pyodbc.Binary()` to explicitly convert to VARBINARY
        cursor.execute("INSERT INTO dbo.QR (name, id, qr) VALUES (?, ?, ?)", 
                       (name, four_digit_number, pyodbc.Binary(qr_binary)))

        conn.commit()
        conn.close()
        print("✅ Data inserted successfully!")

    except Exception as e:
        print("❌ Error:", e)

# Example usage
name = input("Enter name: ")
four_digit_number = input("Enter a 4-digit number: ")

if four_digit_number.isdigit() and len(four_digit_number) == 4:
    insert_data(name, int(four_digit_number))
else:
    print("❌ Invalid input! Please enter a 4-digit number.")


✅ Data inserted successfully!


In [ ]:
import pyodbc
import io

# Database connection parameters
server = 'DESKTOP-7L2TS2S\\SQLEXPRESS'
database = 'Church'
driver = '{ODBC Driver 17 for SQL Server}'

conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

def fetch_qr_info(four_digit_number):
    """Fetch name and QR code from the database and display it."""
    try:
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()

        # Fetch the record where ID matches
        cursor.execute("SELECT name, qr FROM dbo.QR WHERE id = ?", (four_digit_number,))
        row = cursor.fetchone()

        if row:
            name, qr_binary = row
            print(f"✅ Name: {name}")
            print(f"✅ ID: {four_digit_number}")

            if qr_binary:
                # Convert binary QR code back to an image
                qr_image = Image.open(io.BytesIO(qr_binary))
                qr_image.show()
            else:
                print("❌ QR code not found in database.")

        else:
            print("❌ No record found for this ID.")

        conn.close()
    except Exception as e:
        print("❌ Error:", e)

# Example usage
four_digit_number = input("Enter a 4-digit number to retrieve info: ")

if four_digit_number.isdigit() and len(four_digit_number) == 4:
    fetch_qr_info(int(four_digit_number))
else:
    print("❌ Invalid input! Please enter a 4-digit number.")


✅ Name: maria                                             
✅ ID: 1234
